<a href="https://colab.research.google.com/github/aprpure/faster-whisper-youtube/blob/main/faster_whisper_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Youtube Videos Transcription with Faster Whisper**




[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/lewangdev/whisper-youtube/blob/main/faster_whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/lewangdev/faster_whisper_youtube)](https://github.com/lewangdev/faster_whisper_youtube)


[faster-whisper](https://github.com/guillaumekln/faster-whisper) is a reimplementation of OpenAI's Whisper model using CTranslate2, which is a fast inference engine for Transformer models.

This implementation is up to 4 times faster than openai/whisper for the same accuracy while using less memory. The efficiency can be further improved with 8-bit quantization on both CPU and GPU.

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

This Notebook will guide you through the transcription of a Youtube video using Faster Whisper. You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript and video audio in your Google Drive.

# **Check GPU type** 🕵️


In [ ]:

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

# **Install libraries** 🏗️

In [ ]:
#@markdown This cell will take a little while to download several libraries, including Faster Whisper.

#@markdown ---
# ! apt install libcublas11
! pip install faster-whisper
! pip install yt-dlp

import sys
import warnings
from faster_whisper import WhisperModel
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

# **Optional:** Save data in Google Drive 💾


In [ ]:
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive.

# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
#@markdown ---
drive_path = "Colab Notebooks/Faster Whisper Youtube" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change your Google Drive path.**

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


# **Model selection** 🧠


In [ ]:

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~0.8 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1.0 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~1.4 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~2.7 GB     |      ~2x       |
#@markdown | large-v1  |   1550 M   |        N/A         |      `large-v1`       |    ~4.3 GB     |       1x       |
#@markdown | large-v2  |   1550 M   |        N/A         |      `large-v2`       |    ~4.3 GB     |       1x       |
#@markdown | large-v3  |   1550 M   |        N/A         |      `large-v3`       |    ~4.3 GB     |       1x       |

#@markdown ---
model_size = 'large-v2' #@param ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large-v1', 'large-v2', 'large-v3']
device_type = "cuda" #@param {type:"string"} ['cuda', 'cpu']
compute_type = "float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

model = WhisperModel(model_size, device=device_type, compute_type=compute_type)


# **Video selection** 📺


In [ ]:
#@markdown Enter the URL of the Youtube video you want to transcribe, wether you want to save the audio file in your Google Drive, and run the cell.

import json
import os
Type = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive']
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "kyu/" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

video_path_local_list = []

if Type == "Youtube video or playlist":

    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'm4a',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        list_video_info = [ydl.extract_info(URL, download=True)]
        # error_code = ydl.download([URL])

    for video_info in list_video_info:
        if 'entries' in video_info:
            list_video_entries = video_info['entries']
            for entry in list_video_entries:
                video_path_local_list.append(Path(f"{entry['title']}.m4a"))
        else:
            video_path_local_list.append(Path(f"{video_info['title']}.m4a"))

elif Type == "Google Drive":
    # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
    video_path = drive_mount_path / Path(video_path.lstrip("/"))
    if video_path.is_dir():
        for video_path_drive in video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / (video_path_drive.name)
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif video_path.is_file():
        video_path_local = Path(".").resolve() / (video_path.name)
        shutil.copy(video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(video_path)} does not exist.**"))

else:
    raise(TypeError("Please select supported input type."))

# print(list_video_info[0]['entries'][0])
for video_path_local in video_path_local_list:
    print(video_path_local)
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])

# **Run the model** 🚀


In [ ]:
#@markdown Run this cell to execute the transcription of the video. This can take a while and vary based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown #### Language
language = "zh" #@param ["auto", "en", "zh", "ja", "fr", "de"] {allow-input: true}
#@markdown #### initial prompt
initial_prompt = "Please do not translate, only transcription is allowed. Here are  some Chinese words you might need: 爆破" #@param {type:"string"}
#@markdown ---
#@markdown #### Word-level timestamps
word_level_timestamps = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### VAD filter
vad_filter = True #@param {type:"boolean"}
vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
#@markdown ---
#@markdown #### Output(Default is srt, txt if `text_only` be checked )
text_only = False #@param {type:"boolean"}

def seconds_to_time_format(s):
    # Convert seconds to hours, minutes, seconds, and milliseconds
    hours = s // 3600
    s %= 3600
    minutes = s // 60
    s %= 60
    seconds = s // 1
    milliseconds = round((s % 1) * 1000)

    # Return the formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"

for video_path_local in video_path_local_list:
    print(f"Processing video: {video_path_local}")  # 输出当前 video_path_local
    segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                      language=None if language == "auto" else language,
                                      initial_prompt=initial_prompt,
                                      word_timestamps=word_level_timestamps,
                                      vad_filter=vad_filter,
                                      vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

    display(Markdown(f"Detected language '{info.language}' with probability {info.language_probability}"))

    ext_name = '.txt' if text_only else ".srt"
    transcript_file_name = video_path_local.stem + ext_name
    sentence_idx = 1
    with open(transcript_file_name, 'w') as f:
        for segment in segments:
            if word_level_timestamps:
                for word in segment.words:
                    ts_start = seconds_to_time_format(word.start)
                    ts_end = seconds_to_time_format(word.end)
                    print(f"[{ts_start} --> {ts_end}] {word.word}")
                    if not text_only:
                        f.write(f"{sentence_idx}\n")
                        f.write(f"{ts_start} --> {ts_end}\n")
                        f.write(f"{word.word}\n\n")
                    else:
                        f.write(f"{word.word}")
                    f.write("\n")
                    sentence_idx = sentence_idx + 1
            else:
                ts_start = seconds_to_time_format(segment.start)
                ts_end = seconds_to_time_format(segment.end)
                print(f"[{ts_start} --> {ts_end}] {segment.text}")
                if not text_only:
                    f.write(f"{sentence_idx}\n")
                    f.write(f"{ts_start} --> {ts_end}\n")
                    f.write(f"{segment.text.strip()}\n\n")
                else:
                    f.write(f"{segment.text.strip()}\n")
                sentence_idx = sentence_idx + 1

    try:
        shutil.copy(video_path_local.parent / transcript_file_name, drive_whisper_path / transcript_file_name)
        display(Markdown(f"**Transcript file created: {drive_whisper_path / transcript_file_name}**"))
    except:
        display(Markdown(f"**Transcript file created: {video_path_local.parent / transcript_file_name}**"))


Processing video: 周易 八卦 林武樟  梅花易数  教学 01（持续更新）.m4a


Detected language 'zh' with probability 1

[00:00:01,750 --> 00:00:29,800] 今天是我们梅花易数职业班的第一堂课
[00:00:30,800 --> 00:00:33,800] 梅花易数它是一种占卜的方式
[00:00:34,800 --> 00:00:37,800] 占卜的方式跟种类非常的多
[00:00:38,800 --> 00:00:41,800] 我们可以待会用点时间跟各位做个分析
[00:00:42,800 --> 00:00:46,800] 我们第一堂课里面我们有一些观念跟想法
[00:00:46,800 --> 00:00:48,800] 必须要跟各位做分享
[00:00:49,800 --> 00:00:55,800] 这个部分对于将来各位在以梅花易数这种占卜方式从事职业的时候
[00:00:56,800 --> 00:01:00,800] 对于你的思考模式跟你的占卦其实是有帮助的
[00:01:01,800 --> 00:01:03,800] 如果说我们在谈的过程当中
[00:01:04,800 --> 00:01:08,800] 可能有的部分涉及到一些所谓专业的名词
[00:01:09,800 --> 00:01:11,800] 可能当下你听得不是很懂
[00:01:12,800 --> 00:01:13,800] 那没有关系
[00:01:14,800 --> 00:01:15,800] 等你后面课程继续上
[00:01:16,800 --> 00:01:18,800] 再从头来看我们这个教学课程
[00:01:19,800 --> 00:01:20,800] 你就会了解
[00:01:21,800 --> 00:01:25,800] 而且对你的整个判卦的一个做法跟你的想法
[00:01:26,800 --> 00:01:29,800] 其实它是有一些可以参考作用的
[00:01:30,800 --> 00:01:33,800] 我们在以往的课程里面像扬载风水
[00:01:34,800 --> 00:01:36,800] 像包括学八字命理
[00:01:37,800 --> 00:01:41,800] 很多部分我们都希望各位学好之后是从事教学
[00:01:42,800 --> 00:01:43,800] 不是从事职业
[00:01:43,800 -->

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Faster Whisper Youtube/周易 八卦 林武樟  梅花易数  教学 01（持续更新）.srt**

Processing video: 周易 八卦 林武樟 梅花易数 教学 02（持续更新）.m4a


...

[00:00:00,750 --> 00:00:06,360] 第十九堂課
[00:00:54,720 --> 00:01:04,720] 第十九堂課
[00:01:24,720 --> 00:01:34,720] 第十九堂課
[00:01:54,720 --> 00:02:21,670] 只要我們在補掛當中,只要在時間點,剛好整點上,我們忠於學術的範圍裡面,它終究不能得掛,所以不管它是第一個、第二個還是第三個。
[00:02:21,670 --> 00:02:41,670] 那像這種情況,其實我都有一個建議跟我們的做法,你今天同一件事情,要問補三個掛,你先不要解,你就先把三個都抓出來,比如說你用三張紙,去A家發展如何,抓好了,數字寫好了,先不要起。
[00:02:42,670 --> 00:03:02,670] 然後再第二個,接著第三個,三個全部都起好了,它的時間點就落差在幾分鐘而已,那絕對不會掛到整點,它會掛到整點是因為你可能在解第一個掛,還是你在解第二個掛的時候,花了相當多的時間,不小心把它掛到整點的時間點上。
[00:03:03,670 --> 00:03:24,180] 那當然,會如此做也有它的機,會這樣子也有它的機。那如果說萬一它補了兩個掛,這兩個掛解出來了,第三個掛發現到了整點,那怎麼辦?第三個掛不得掛了,這家公司不給它建議了。
[00:03:25,180 --> 00:03:50,180] 我們不給它建議,這家公司可能有很多不可跟它講的一些理由跟原因,它自己去判斷。其實求職的掛來說,求職的掛來講,很多大部分在我們的經驗裡面,在問掛有關於求職這方面,有關於換工這方面,絕大部分的求專者的心理都已經有了決定,
[00:03:51,180 --> 00:04:07,180] 它其實有想法了,它只是希望透過掛項來支持它的想法,它只是希望透過掛項來給它的想法更為圓滿跟確定一點而已。所以依照我們的經驗,絕大部分它其實想去哪家公司,它心裡有底。
[00:04:07,180 --> 00:04:26,180] 所以你開始慢慢跟它談的過程當中,你就會發現,它好像有透露出它比較喜歡哪一家。那特別喜歡哪一家,我們看它的掛項,如果說並不是很兇,事實上發展還可以,我們根據那個掛跟它多談。
[00:04:27,180 --> 00:04:42,180] 如果說這幾個掛,三個掛,我

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Faster Whisper Youtube/周易 八卦 林武樟 梅花易数 教学 19（持续更新）.srt**

Processing video: 周易 八卦 林武樟 梅花易数 教学 20（持续更新）.m4a


Detected language 'zh' with probability 1

[00:00:01,750 --> 00:00:28,820] 我们继续上我们第二次堂课
[00:00:28,820 --> 00:00:31,820] 我们刚刚提到所谓学术的一个合理性
[00:00:31,820 --> 00:00:33,820] 你像洋宅风水来讲
[00:00:33,820 --> 00:00:35,820] 很多人很想问说
[00:00:35,820 --> 00:00:37,820] 你是哪一派的?
[00:00:37,820 --> 00:00:39,820] 你是哪一派的?
[00:00:39,820 --> 00:00:40,820] 我不知道为什么
[00:00:40,820 --> 00:00:44,820] 现代人那么喜欢把它化婚派别
[00:00:44,820 --> 00:00:47,820] 其实洋宅它只有做法而已
[00:00:47,820 --> 00:00:49,820] 那做法当然在很多书上
[00:00:49,820 --> 00:00:50,820] 尤其现在的人
[00:00:50,820 --> 00:00:53,820] 因为出书的一个方便
[00:00:53,820 --> 00:00:58,460] 很多老师借由出书
[00:00:58,460 --> 00:01:00,460] 打开自己的知名度
[00:01:00,460 --> 00:01:02,460] 这是他们的一个管道之一
[00:01:02,460 --> 00:01:04,460] 那为了他要内容跟人家不同
[00:01:04,460 --> 00:01:06,460] 他会去编撰
[00:01:06,460 --> 00:01:07,460] 那编撰没有关系
[00:01:07,460 --> 00:01:09,460] 或许他的经验所得也没关系
[00:01:09,460 --> 00:01:11,460] 但是我们刚刚讲过了
[00:01:11,460 --> 00:01:17,460] 一个合理的一个依据是很重要的
[00:01:17,460 --> 00:01:19,460] 你像比如讲说
[00:01:19,460 --> 00:01:21,460] 现在洋宅风水里面
[00:01:21,460 --> 00:01:25,4

In [ ]:
! rm -rf *.txt
! rm -rf *.m4a